# 爬蟲(股價量)

In [1]:
#https://aronhack.com/zh/retrieve-stock-historical-data-with-python-and-yahoo-finance-api/
#https://blog.techbridge.cc/2020/09/21/python-pandas-zen-tutorial/
import requests
import numpy as np
import yfinance as yf
import pandas as pd
import time
import urllib.request as request
import json
stock_list=['2603']
# 開始爬蟲
historical_data = pd.DataFrame()
for i in stock_list:    
    
    # 抓取股票資料
    stock_id = i + '.TW'
    data = yf.Ticker(stock_id)
    df = data.history(period="max")

    # 在表格中增加股票代號
    df['STOCK_ID'] = i
    
    # 將資料格式化
    stock = pd.DataFrame(df)

    # 將df與一開始的空表格合併成一張完整表格
    historical_data = pd.concat([historical_data, stock])
    
    # 記得爬蟲禮儀
    time.sleep(0.8)
# 挑選股票與時段
def pick(stock_number, period_start, period_end):
    global historical_data
    a = historical_data['STOCK_ID'] == stock_number
    a = historical_data[a].loc[period_start : period_end]
    return a

# 用函式定義標的
s2603 = pick('2603', '2021-01-01', '2021-07-06')

# Abstract API 

In [2]:
# Abstract API能直接讀取資料，不須額外的程式碼抓資料。
# 前提: 資料必須是Abstract API看得懂的格式

In [3]:
# Function API：
import talib

# Abstract API：
from talib import abstract

In [4]:
# Function API：
talib.ADX(s2603['High'], s2603['Low'], s2603['Close'], timeperiod = 14)

Date
2021-01-04          NaN
2021-01-05          NaN
2021-01-06          NaN
2021-01-07          NaN
2021-01-08          NaN
                ...    
2021-06-30    42.479972
2021-07-01    44.151752
2021-07-02    45.770499
2021-07-05    46.989281
2021-07-06    48.475669
Length: 119, dtype: float64

In [5]:
# Abstract API：

# Abstract API 只辨別小寫，所以先轉換大小寫
s2603.columns=['open','high','low','close','volume', 'dividends', 'stock splits','stock_id']

abstract.ADX(s2603, timperiod = 14)

Date
2021-01-04          NaN
2021-01-05          NaN
2021-01-06          NaN
2021-01-07          NaN
2021-01-08          NaN
                ...    
2021-06-30    42.479972
2021-07-01    44.151752
2021-07-02    45.770499
2021-07-05    46.989281
2021-07-06    48.475669
Length: 119, dtype: float64

# 利用迴圈處理158種指標

In [6]:
# 確認價量資料表 df 的值都是 float 格式
df = df.astype('float')

# 準備一份想要計算並且併入 df 的技術指標清單
ta_list = talib.get_functions()

In [10]:
# 嘗試處理每項指標，若無法計算則列印出
for x in ta_list:
    try:
        # x 為技術指標的代碼，透過迴圈填入，再透過 eval 計算出 output  #eval:執行括弧內之字串
        output = eval('abstract.'+x+'(s2603)')
        # 如果輸出是一維資料，幫這個指標取名為 x 本身；多維資料則不需命名
        output.name = x.lower() if type(output) == pd.core.series.Series else None
        # 透過 merge 把輸出結果併入 df DataFrame
        s2603 = pd.merge(s2603, pd.DataFrame(output), left_on = s2603.index, right_on = output.index)
        s2603 = s2603.set_index('key_0')
    except:
        print(x)           #MAVP可以自由改變每一期計算移動平均的天數，所以須額外準備『periods』的資料，把各期天數放進去。

MAVP


In [11]:
# 共157種指標數值已自動納入資料
s2603

,open,high,low,close,volume,dividends,stock splits,stock_id,ht_dcperiod_x,ht_dcphase_x,...,linearreg_slope_y,stddev_y,tsf_y,var_y,atr_y,natr_y,trange_y,ad_y,adosc_y,obv_y
key_0,,,,,,,,,,,,,,,,,,,,,
2021-01-04,42.349998,44.750000,42.150002,44.750000,513626515,0.0,0.0,2603,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.136265e+08,NaN,5.136265e+08
2021-01-05,45.000000,45.500000,43.150002,45.500000,533867334,0.0,0.0,2603,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.349998,1.047494e+09,NaN,1.047494e+09
2021-01-06,44.750000,46.200001,42.250000,42.549999,564149052,0.0,0.0,2603,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.950001,5.690381e+08,NaN,4.833448e+08
2021-01-07,43.299999,43.799999,38.299999,38.450001,775697280,0.0,0.0,2603,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.500000,-1.643480e+08,NaN,-2.923525e+08
2021-01-08,38.500000,40.750000,37.200001,40.299999,548579386,0.0,0.0,2603,NaN,NaN,...,NaN,2.650547,NaN,7.025399,NaN,NaN,3.549999,2.451547e+08,NaN,2.562269e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-30,181.500000,197.000000,181.500000,197.000000,124635805,0.0,0.0,2603,23.867542,178.600748,...,5.090110,15.730226,187.604396,247.440000,11.465591,5.820097,17.500000,1.106256e+09,2.397896e+08,1.131819e+10
2021-07-01,204.000000,216.000000,195.000000,213.000000,164144137,0.0,0.0,2603,24.335307,173.063412,...,5.721978,18.243903,199.450549,332.840000,12.146621,5.702639,21.000000,1.223502e+09,2.634911e+08,1.148233e+10
2021-07-02,213.000000,219.000000,200.000000,206.500000,543116488,0.0,0.0,2603,25.483114,170.481519,...,6.076923,14.558159,208.005495,211.940000,12.636148,6.119200,19.000000,1.051992e+09,1.946620e+08,1.093922e+10


In [23]:
for i in s2603.columns:
    print(i)

open
high
low
close
volume
dividends
stock splits
stock_id
ht_dcperiod_x
ht_dcphase_x
inphase_x
quadrature_x
sine_x
leadsine_x
ht_trendmode_x
add_x
div_x
max_x
maxindex_x
min_x
minindex_x
min_y
max_y
minidx_x
maxidx_x
mult_x
sub_x
sum_x
acos_x
asin_x
atan_x
ceil_x
cos_x
cosh_x
exp_x
floor_x
ln_x
log10_x
sin_x
sinh_x
sqrt_x
tan_x
tanh_x
adx_x
adxr_x
apo_x
aroondown_x
aroonup_x
aroonosc_x
bop_x
cci_x
cmo_x
dx_x
macd_x
macdsignal_x
macdhist_x
macd_y
macdsignal_y
macdhist_y
macd_x
macdsignal_x
macdhist_x
mfi_x
minus_di_x
minus_dm_x
mom_x
plus_di_x
plus_dm_x
ppo_x
roc_x
rocp_x
rocr_x
rocr100_x
rsi_x
slowk_x
slowd_x
fastk_x
fastd_x
fastk_y
fastd_y
trix_x
ultosc_x
willr_x
upperband_x
middleband_x
lowerband_x
dema_x
ema_x
ht_trendline_x
kama_x
ma_x
mama_x
fama_x
midpoint_x
midprice_x
sar_x
sarext_x
sma_x
t3_x
tema_x
trima_x
wma_x
cdl2crows_x
cdl3blackcrows_x
cdl3inside_x
cdl3linestrike_x
cdl3outside_x
cdl3starsinsouth_x
cdl3whitesoldiers_x
cdlabandonedbaby_x
cdladvanceblock_x
cdlbelthold_x
cdl

In [25]:
s2603[['upperband_x','middleband_x','lowerband_x']]

,upperband_x,middleband_x,lowerband_x
key_0,,,
2021-01-04,NaN,NaN,NaN
2021-01-05,NaN,NaN,NaN
2021-01-06,NaN,NaN,NaN
2021-01-07,NaN,NaN,NaN
2021-01-08,47.611094,42.31,37.008906
...,...,...,...
2021-06-30,204.360451,172.90,141.439549
2021-07-01,221.587806,185.10,148.612194
2021-07-02,223.516318,194.40,165.283682
